In [1]:
import matplotlib
matplotlib.use('TkAgg')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from new_datasets_py import create_subsets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import BalancedRandomForestClassifier


In [2]:
data = pd.read_csv('crypto-markets.csv')
filtered_data = data[data['ranknow'] < 30]
filtered_data.loc[:, 'date'] = pd.to_datetime(filtered_data['date'])
filtered_data.set_index('date', inplace=True)
datasets_with_labels = []

# Group the filtered data by 'slug'
grouped = filtered_data.groupby('slug')

# Iterate over each group
for crypto, group in grouped:
    close_values = group['close'].values

    # Iterate over the windows of 10 days
    for start in range(len(close_values) - 9):
        end = start + 10
        window = close_values[start:end]

        # Calculate label based on values at day 7 and day 10
        value_day_7 = window[6]  # Index 6 corresponds to the 7th day (0-indexed)
        value_day_10 = window[9]  # Index 9 corresponds to the 10th day (0-indexed)
        label = 1 if value_day_10 > value_day_7 else 0

        # Append the dataset and label as a tuple to the datasets_with_labels list
        datasets_with_labels.append((window, label))

# Convert the list of tuples to a DataFrame
combined_table = pd.DataFrame(datasets_with_labels, columns=['close_values', 'label'])

# Print to check
print(combined_table)
missing_values = combined_table.isnull().sum()
print(missing_values)

                                            close_values  label
0      [0.695589, 0.742796, 0.86392, 0.734774, 1.07, ...      0
1      [0.742796, 0.86392, 0.734774, 1.07, 1.43, 1.33...      1
2      [0.86392, 0.734774, 1.07, 1.43, 1.33, 1.4, 1.4...      1
3      [0.734774, 1.07, 1.43, 1.33, 1.4, 1.4, 1.31, 2...      1
4      [1.07, 1.43, 1.33, 1.4, 1.4, 1.31, 2.38, 3.18,...      1
...                                                  ...    ...
21966  [0.125544, 0.122172, 0.119888, 0.117055, 0.100...      1
21967  [0.122172, 0.119888, 0.117055, 0.100511, 0.112...      1
21968  [0.119888, 0.117055, 0.100511, 0.112595, 0.111...      1
21969  [0.117055, 0.100511, 0.112595, 0.11143, 0.1131...      1
21970  [0.100511, 0.112595, 0.11143, 0.113184, 0.1123...      1

[21971 rows x 2 columns]
close_values    0
label           0
dtype: int64


d:\Projects\time-stamps-predicition\venv\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [3]:
# Checking balance of the dataset
print(combined_table['label'].value_counts())
combined_table['label'].value_counts().plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()
class_counts = combined_table['label'].value_counts()
imbalance_ratio = class_counts.min() / class_counts.max()
print(f"Imbalance Ratio: {imbalance_ratio}")

label
0    11709
1    10262
Name: count, dtype: int64
Imbalance Ratio: 0.8764198479801861


In [4]:
X = []
y = []
X = combined_table['close_values'].apply(lambda x: x[:7]).tolist()
y = combined_table['label'].astype(int).tolist()
X = np.array(X)
y = np.array(y)


In [ ]:
classifiers = {
    "Multi-Layer Perceptron": MLPClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)

for clf_name, clf in classifiers.items():
    print(f"Evaluating {clf_name}:")
    for train, test in rskf.split(X, y):
        model = clf
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        acc = balanced_accuracy_score(y_pred, y[test])

        print(f"   Balanced Accuracy: {acc}")


In [6]:
classifiers = {
    "Multi-Layer Perceptron": MLPClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Balanced Random Forest": BalancedRandomForestClassifier(replacement=True, sampling_strategy='all', random_state=42),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
results = {clf_name: [] for clf_name in classifiers}

for clf_name, clf in classifiers.items():
    print(f"Evaluating {clf_name}:")
    for train, test in rskf.split(X, y):
        model = clf
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        acc = balanced_accuracy_score(y[test], y_pred)
        results[clf_name].append(acc)

        print(f"   Balanced Accuracy: {acc}")

fig, ax = plt.subplots(figsize=(10, 6))
ax.boxplot([results[clf] for clf in classifiers], labels=classifiers.keys())
ax.set_title('Balanced Accuracy Scores of Classifiers')
ax.set_xlabel('Classifiers')
ax.set_ylabel('Balanced Accuracy')
plt.xticks(rotation=45)
plt.show()

for clf_name, scores in results.items():
    print(f"{clf_name}: Mean = {np.mean(scores):.4f}, Std = {np.std(scores):.4f}")


Evaluating Multi-Layer Perceptron:
   Balanced Accuracy: 0.5054657469458995
   Balanced Accuracy: 0.5076308048801195
   Balanced Accuracy: 0.5210048974319278
   Balanced Accuracy: 0.5292166689139587
   Balanced Accuracy: 0.5022441398622118
   Balanced Accuracy: 0.5066503040893687
   Balanced Accuracy: 0.5045395298665108
   Balanced Accuracy: 0.5395602465695504
   Balanced Accuracy: 0.5219483439122524
   Balanced Accuracy: 0.5213453894686992
Evaluating Decision Tree:
   Balanced Accuracy: 0.5505111554896855
   Balanced Accuracy: 0.5452211335340915
   Balanced Accuracy: 0.5449728909996787
   Balanced Accuracy: 0.5460517992485721
   Balanced Accuracy: 0.5574904917174583
   Balanced Accuracy: 0.5456827670489501
   Balanced Accuracy: 0.5421417192283299
   Balanced Accuracy: 0.5454300484582745
   Balanced Accuracy: 0.5489085651789594
   Balanced Accuracy: 0.5431812001190993
Evaluating Random Forest:
   Balanced Accuracy: 0.5771977481455353
   Balanced Accuracy: 0.5643865808367583
   Balanced

d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


   Balanced Accuracy: 0.5808123580788025


d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


   Balanced Accuracy: 0.5723191054778991


...

   Balanced Accuracy: 0.570606585730861


d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


   Balanced Accuracy: 0.5885262425444007


d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
d:\Projects\time-stamps-predicition\venv\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


   Balanced Accuracy: 0.5555228769933374
Evaluating Logistic Regression:
   Balanced Accuracy: 0.5082283617359445
   Balanced Accuracy: 0.5141082079427624
   Balanced Accuracy: 0.519967605701796
   Balanced Accuracy: 0.5211232964278045
   Balanced Accuracy: 0.5019818675246922
   Balanced Accuracy: 0.5176396167654508
   Balanced Accuracy: 0.5061856712661243
   Balanced Accuracy: 0.5191067680112131
   Balanced Accuracy: 0.5196960579168768
   Balanced Accuracy: 0.5068305246299838
Evaluating Support Vector Machine:
   Balanced Accuracy: 0.5041544252376082
   Balanced Accuracy: 0.5049943984165747
